In [1]:
import PyPDF2

In [2]:
pdf_file = "../data/LLM-Overview.pdf"
pdf_reader = PyPDF2.PdfReader(pdf_file)
pdf_reader.pages[20].extract_text()

'we need to augment the models through some external means.\nIn this section we first provide a brief overview of the main\nshortcoming of LLMs, with a deeper look at the issue of\nhallucination. We then describe how prompting and some aug-\nmentation approaches can not only address those limitations\nbut also be used to augment the capabilities of LLMs going\nas far as turning an LLM into a full-blown AI agent with the\nability to interface with the external world.\nA. LLM limitations\nIt is important to remember that LLMs are trained to predict\na token. While fine-tuning and alignment improves their per-\nformance and adds different dimensions to their abilities, there\nare still some important limitations that come up, particularly\nif they are used naively. Some of them include the following:\n• They don’t have state/memory. LLMs on their own\ncannot remember even what was sent to them in the\nprevious prompt. That is an important limitation for\nmany of the uses cases that requir

In [3]:
from langchain_core.documents import Document

# Split up the PDF document into Pages.
docs = []
for page_index, page in enumerate(pdf_reader.pages):
        docs.append(Document(page_content=page.extract_text(), meta_data={'file': pdf_file, 'page': page_index}))


In [4]:
docs[6]

Document(page_content='GPT Family PaLM Family    LLaMA  1/2 Family\nGPT\nGPT1\nGPT2\nGPT3\nGPT4\nGPT3.5 Turbo\ntext-davinci\ncode-davinci\nCODEX\nInstructGPT\nWebGPT\nGPT4 V ision\nGPT4 Turbo\nGorilla\nMistral\nVigogne\nStable Beluga2\nKoala\nCode LLaMA\nVicuna\n Alpaca\nBaize\nLong LLaMA\nGiraf fe\nGuanaco\nTulu\nWizardLM\nMed-PaLM\nPaLM-E\n Med-PaLM2\nFLAN-PaLM\n U-PaLM\nPaLM2\nPaLM\nFig. 8: Popular LLM Families.\nFig. 9: GPT-3 shows that larger models make increasingly\nefficient use of in-context information. It shows in-context\nlearning performance on a simple task requiring the model to\nremove random symbols from a word, both with and without\na natural language task description. Courtesy of [56].\n(RLHF), as shown in 10. The resultant InstructGPT models\nhave shown improvements in truthfulness and reductions in\ntoxic output generation while having minimal performance\nregressions on public NLP datasets.\nFig. 10: The high-level overview of RLHF. Courtesy of [59].\nThe most im

In [5]:
docs[0].page_content

'Large Language Models: A Survey\nShervin Minaee, Tomas Mikolov, Narjes Nikzad, Meysam Chenaghlu\nRichard Socher, Xavier Amatriain, Jianfeng Gao\nAbstract —Large Language Models (LLMs) have drawn a\nlot of attention due to their strong performance on a wide\nrange of natural language tasks, since the release of ChatGPT\nin November 2022. LLMs’ ability of general-purpose language\nunderstanding and generation is acquired by training billions of\nmodel’s parameters on massive amounts of text data, as predicted\nby scaling laws [1], [2]. The research area of LLMs, while very\nrecent, is evolving rapidly in many different ways. In this paper,\nwe review some of the most prominent LLMs, including three\npopular LLM families (GPT, LLaMA, PaLM), and discuss their\ncharacteristics, contributions and limitations. We also give an\noverview of techniques developed to build, and augment LLMs.\nWe then survey popular datasets prepared for LLM training,\nfine-tuning, and evaluation, review widely us

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
)

# Build up a list of Documents, each holding a Paragraph from a Page.
# Meta-data identifies the source and the page number of the Document.
doc_texts = []
for pindex, page in enumerate(docs):    
    doc_texts.extend(text_splitter.create_documents(texts=[page.page_content], metadatas=[{'file': pdf_file, 'page': pindex}]))


In [7]:
doc_texts[200].metadata

{'file': '../data/LLM-Overview.pdf', 'page': 8}

In [8]:
from qutils import VectorStore

# Prepare for our persistent Vector Store if non-existent.
# Else open the existing Vector Store.
v = VectorStore(persist_directory="DocsDB")

In [9]:
v.persist_directory

'DocsDB'

In [18]:
# If no Vector Store exist, create one and load the documents.
if not v:
    v.store_documents(doc_texts)

In [12]:
# Search for text chunks similar to our Query.
answer = v.similarity_search(
    query="Why is the Transformer so successful",
    num_results=3
)

In [16]:
# Find out what Pages the text chunks are referring to.
s = set()
for a in answer:
    s.add(a.metadata['page'])
    print(a.metadata['page'])

11
11
39


In [17]:
s

{11, 39}